In [1]:
from lcls_live.datamaps.tabular import TabularDataMap, datamap_from_tao_data
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station, subbooster_pvinfo, SUBBOOSTER_SECTORS

from scipy.constants import e as e_charge
from pytao import Tao
import json
import os
from lcls_live import data_dir
import pandas as pd

# cu_hxr and cu_sxr

## BPMs



In [2]:
def build_bpm_dm(tao, model):
    """
    Build BPM datamaps
    """
    
    if model == 'cu_hxr':
        suffix = 'CUHTH' # Ten Hz
    elif model == 'cu_sxr':
        suffix = 'CUSTH' # Ten Hz
    elif model == 'cu_spec':
        suffix = '1H'
    
    dm_x = datamap_from_tao_data(tao, 'orbit', 'x', tao_factor = .001, pv_attribute=':X'+suffix)
    dm_y = datamap_from_tao_data(tao, 'orbit', 'y', tao_factor = .001, pv_attribute=':Y'+suffix)    
    
    dm_charge = datamap_from_tao_data(tao, 'orbit', 'charge', tao_factor = e_charge, pv_attribute=':TMIT'+suffix)
    
    dm_x.data.append(dm_y.data, ignore_index = True)
    dm_x.data.append(dm_charge.data, ignore_index = True)    
    
    frames = [dm_x.data, dm_y.data, dm_charge.data]
    
    dm_x.data = pd.concat(frames, ignore_index=True)
    
    return dm_x



## Build datamap for linac

In [3]:
def build_linac_dm(model):
    dat0 = [
        
        {'name': 'BC1_offset',
         'pvname':'BMLN:LI21:235:MOTR',  # mm
         'bmad_factor': 0.001,
         'bmad_name': 'O_BC1_OFFSET',
         'bmad_attribute': 'offset'
        },
        
        {'name': 'BC2_offset',
         'pvname':'BMLN:LI24:805:MOTR', # mm
         'bmad_factor': 0.001,
         'bmad_name': 'O_BC2_OFFSET',
         'bmad_attribute': 'offset'
        },
        
        {
        'name': 'L1_phase',
        'description': 'Controls the L1 phase, which is the single klystron L21_1. We will disable this for now, because the KlystronDataMap handles the phase directly.',
        'pvname': 'ACCL:LI21:1:L1S_S_PV',
        'bmad_name':'O_L1',
        'bmad_factor': 0,  # We'll disable this for now. The Klystron handles it. 
        'bmad_attribute':'phase_deg'
        
    }
    ]
    
    dat_hxr = [
            {
        'name': 'L2_phase',
        'pvname': 'ACCL:LI22:1:PDES',
        'bmad_name':'O_L2',
        'bmad_factor': 1,
        'bmad_attribute':'phase_deg'
        
    },
        {
        'name': 'L3_phase',
        'pvname': 'ACCL:LI25:1:PDES',
        'bmad_name':'O_L3',
        'bmad_attribute':'phase_deg',
        'bmad_offset': 0
        
    }, 
    ]
    
    # SXR has different PVs
    dat_sxr = [
            {
        'name': 'L2_phase',
        'pvname': 'ACCL:LI22:1:PDES:SETDATA_1',
        'bmad_name':'O_L2',
        'bmad_factor': 1,
        'bmad_attribute':'phase_deg'
        
    },
        {
        'name': 'L3_phase',
        'pvname': 'ACCL:LI25:1:PDES:SETDATA_1',
        'bmad_name':'O_L3',
        'bmad_attribute':'phase_deg',
        'bmad_offset': 0
        
    }, 
    ]

    #Note that there are sone NaNs here. That's okay.
    if model == 'cu_hxr':
        df = pd.DataFrame(dat0+dat_hxr)
    elif model == 'cu_sxr':
        df = pd.DataFrame(dat0+dat_sxr)
    else:
        raise ValueError(f'Unknown model: {model}')
        
    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor', offset='bmad_offset')
    
    return dm

## Build datamaps for klystrons

In [4]:
def build_klystron_dms(tao, model):
    
    if model == 'cu_hxr':
        beamcode = 1
    elif model == 'cu_sxr':
        beamcode = 2
    else:
        raise ValueError(f'Unknown model: {model}')
    
    
    klystron_names = tao.lat_list('overlay::K*', 'ele.name', flags='-no_slaves')

    klystron_datamaps = []
    for sector, station in existing_LCLS_klystrons_sector_station:
        info = klystron_pvinfo(sector, station, beamcode=beamcode)
        k = KlystronDataMap(**info)

        if k.name in klystron_names:
            klystron_datamaps.append(k)
    
    return klystron_datamaps

## Build datamaps for quadrupoles

In [5]:
def quad_pvinfo(tao, ele):
    """
    Returns dict of PV information for use in a DataMap
    """
    head = tao.ele_head(ele)
    attrs = tao.ele_gen_attribs(ele)
    device = head['alias']
    
    d = {}
    d['bmad_name'] = ele
    d['pvname_rbv'] = device+':BACT'
    d['pvname'] = device+':BDES'    
    d['bmad_factor'] = -1/attrs['L']/10
    d['bmad_attribute'] = 'b1_gradient'
    return d

def build_quad_dm(tao):
    quad_names = tao.lat_list('quad::*', 'ele.name', flags='-no_slaves')
    dfq = pd.DataFrame([quad_pvinfo(tao, ele) for ele in quad_names])
    dm = TabularDataMap(dfq, pvname='pvname_rbv', element='bmad_name', attribute = 'bmad_attribute', factor='bmad_factor')
    return dm

## Correctors 

In [6]:

def build_corrector_dm(tao):
    xeles = tao.lat_list('hkicker::*', 'ele.name', flags='-array_out -no_slaves')
    yeles = tao.lat_list('vkicker::*', 'ele.name', flags='-array_out -no_slaves')
    eles = xeles + yeles

    df = pd.DataFrame()
    df['bmad_name'] = pd.Series(eles)
    df['pvname'] = [ tao.ele_head(ele)['alias']+':BACT' for ele in df['bmad_name' ] ]
    df['bmad_factor'] = -1/10 # kG*m -> T (with the correct sign)
    df['bmad_attribute'] = 'bl_kick'

    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute', factor='bmad_factor')    
    
    return dm

## Build datamaps for collimators

## Build datamap for energy measurements

In [7]:
def build_energy_dm(model):
    # The syntax is flexible enough to use for getting measurements for Tao
    ENERGY_MEAS0 = [
        {
        'name': 'L1_energy',
        'pvname': 'BEND:LI21:231:EDES', # or EDES
        'tao_datum': 'BC1.energy[1]',        
        'factor': 1e9
        },
        {
        'name': 'L2_energy',
        'pvname': 'BEND:LI24:790:EDES', # or EDES
        'tao_datum': 'BC2.energy[1]',
        'factor': 1e9
        }
    ]
    
    
    ENERGY_MEAS_HXR = [ {
        'name': 'L3_HXR_energy',
        'pvname': 'BEND:DMPH:400:EDES', # or EDES
        'tao_datum': 'L3.energy[2]',
        'factor': 1e9
        } ]
    
    ENERGY_MEAS_SXR = [ {
        'name': 'L3_SXR_energy',
        'pvname': 'BEND:DMPS:400:EDES', # or EDES
        'tao_datum': 'L3.energy[2]',
        'factor': 1e9
        } ]    
    
    if model == 'cu_hxr':
        df = pd.DataFrame(ENERGY_MEAS0 + ENERGY_MEAS_HXR)
    elif model == 'cu_sxr':
         df = pd.DataFrame(ENERGY_MEAS0 + ENERGY_MEAS_SXR)
    else:
        raise ValueError(f'Unknown model: {model}')    
    
    
    dm = TabularDataMap(df, pvname='pvname', element='tao_datum', factor='factor',
                       tao_format = 'set data {element}|meas  = {value}',
                       bmad_format = '! No equivalent Bmad format for: set data {element}|meas  = {value}'
                       )
    return dm

## Build subbooster datamap

In [8]:
def build_subbooster_dm():
    subboosters = []
    for sector in SUBBOOSTER_SECTORS:

        dat = subbooster_pvinfo(sector) 
        dat['bmad_name'] = f'SBST_{sector}'
        dat['bmad_attribute'] = 'phase_deg'
        subboosters.append(dat)
    df = pd.DataFrame(subboosters)    

    # Make the DataMap object, identifying the columns to be used
    dm = TabularDataMap(df, pvname='phase_pvname', element='bmad_name', attribute='bmad_attribute')
    return dm

## Beginning Twiss datamap function

In [9]:
def beginning_meas_twiss_datamap(name, pvprefix):
    dat =  [
    {
    'name': f'{name}_beta_x_meas',
    'pvname': f'{pvprefix}:BETA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_a'
    },
    {
    'name': f'{name}_beta_y_meas',
    'pvname': f'{pvprefix}:BETA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_b'
    },
    {
    'name': f'{name}_alpha_x_meas',
    'pvname': f'{pvprefix}:ALPHA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_a'
    },
    {
    'name': f'{name}_alpha_y_meas',
    'pvname': f'{pvprefix}:ALPHA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_b'
    },    
    ]
    
    df= pd.DataFrame(dat)

    return TabularDataMap(df, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute')

## Utility function for creating map for given model 

In [10]:
def build_json_rep(model):
    #set up tao
    # Basic model with options
    MODEL = model
    OPTIONS = f'-noplot'
    INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'
    tao = Tao(INIT)
    
    
    rep = []
    
    
    dm = build_bpm_dm(tao, model)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    # Correctors
    dm = build_corrector_dm(tao)
    rep.append(
        {
        "name": "correctors",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    
    #subbooster_dm
    dm = build_subbooster_dm()
    rep.append(
        {
        "name": "subboosters",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #linac
    dm = build_linac_dm(model)
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #klystron
    dms = build_klystron_dms(tao, model)
    for dm in dms:
        rep.append(
            {
            "name": dm.name,
            "class": "klystron",
            "data": dm.to_json()
            }
        )
    
    
    #quad
    dm = build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #otr2
    dm = beginning_meas_twiss_datamap('WS02', 'WIRE:IN20:561' )
    rep.append(
        {
        "name": "beginning_WS02",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    
    #energy_meas
    dm = build_energy_dm(model)
    rep.append(
        {
        "name": "tao_energy_measurements",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    return rep

In [11]:
cu_hxr = build_json_rep("cu_hxr")
cu_sxr = build_json_rep("cu_sxr")

# cu_spec (i.e. the injector)

In [12]:
def build_cu_spec_linac_dm():
    dat =  [
    
        {'name': 'L0A_phase',
             'pvname':'ACCL:IN20:300:L0A_PDES', # deg
             'bmad_factor': 1/360,
             'bmad_name': 'L0A',
             'bmad_attribute': 'phi0'
            },
        {'name': 'L0B_phase',
             'pvname':'ACCL:IN20:400:L0B_PDES', # deg
             'bmad_factor': 1/360,
             'bmad_name': 'L0B',
             'bmad_attribute': 'phi0'
            },
        {'name': 'L0A_voltage',
             'pvname':'ACCL:IN20:300:L0A_ADES', # MV
             'bmad_factor': 1e6,
             'bmad_name': 'L0A',
             'bmad_attribute': 'voltage'
            },    
        {'name': 'L0B_voltage',
             'pvname':'ACCL:IN20:400:L0B_ADES', # MV
             'bmad_factor': 1e6,
             'bmad_name': 'L0B',
             'bmad_attribute': 'voltage'
            }
    ]
    df = pd.DataFrame(dat)
    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor')
    return dm


In [13]:
def build_cu_spec_json_rep():
    #set up tao
    model = 'cu_spec'
    options = '-noplot'
    init = f'-init $LCLS_LATTICE/bmad/models/{model}/tao.init {options}'
    
    tao = Tao(init)
    
    
    rep = []
    
    # BPMs
    dm = build_bpm_dm(tao, model)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )     
    
    # Correctors
    dm = build_corrector_dm(tao)
    rep.append(
        {
        "name": "correctors",
        "class": "tabular",
        "data": dm.to_json()
        }
    )        
    
    # quad
    dm =  build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    # Linac
    dm = build_cu_spec_linac_dm()
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    return rep

cu_spec = build_cu_spec_json_rep()

# Dump all to master file

In [14]:
with open(os.path.join(data_dir, 'datamaps_master.json'), "w") as f:
    
    master = {"cu_hxr": cu_hxr,
              "cu_sxr": cu_sxr,
             "cu_spec": cu_spec}
    
    json.dump(master, f)

# Test

In [15]:
from lcls_live.datamaps import get_datamaps

DM = get_datamaps('cu_spec')
DM.keys()

dict_keys(['bpms', 'correctors', 'quad', 'linac'])

In [16]:
DM['correctors'].data

,bmad_name,pvname,bmad_factor,bmad_attribute
0,XC00,XCOR:IN20:121:BACT,-0.1,bl_kick
1,XC01,XCOR:IN20:221:BACT,-0.1,bl_kick
2,XC02,XCOR:IN20:311:BACT,-0.1,bl_kick
3,XC03,XCOR:IN20:341:BACT,-0.1,bl_kick
4,XC04,XCOR:IN20:381:BACT,-0.1,bl_kick
5,XC05,XCOR:IN20:411:BACT,-0.1,bl_kick
6,XC06,XCOR:IN20:491:BACT,-0.1,bl_kick
7,XC07,XCOR:IN20:521:BACT,-0.1,bl_kick
8,XC08,XCOR:IN20:641:BACT,-0.1,bl_kick
9,XCS1,XCOR:IN20:911:BACT,-0.1,bl_kick


In [17]:
DM['linac'].pvlist

['ACCL:IN20:300:L0A_PDES',
 'ACCL:IN20:400:L0B_PDES',
 'ACCL:IN20:300:L0A_ADES',
 'ACCL:IN20:400:L0B_ADES']

In [18]:
MODEL = 'cu_spec'
OPTIONS = '-noplot'
INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'

In [19]:
tao = Tao(INIT)

In [20]:
datamap_from_tao_data(tao, 'orbit', 'x', tao_factor=.001, pv_attribute=':X').data

,pvname,tao_datum,tao_factor,bmad_name
0,BPMS:IN20:221:X,orbit.x[1],0.001,BPM2
1,BPMS:IN20:235:X,orbit.x[2],0.001,BPM3
2,BPMS:IN20:371:X,orbit.x[3],0.001,BPM5
3,BPMS:IN20:425:X,orbit.x[4],0.001,BPM6
4,BPMS:IN20:511:X,orbit.x[5],0.001,BPM8
5,BPMS:IN20:525:X,orbit.x[6],0.001,BPM9
6,BPMS:IN20:581:X,orbit.x[7],0.001,BPM10
7,BPMS:IN20:631:X,orbit.x[8],0.001,BPM11
8,BPMS:IN20:651:X,orbit.x[9],0.001,BPM12
9,BPMS:IN20:925:X,orbit.x[10],0.001,BPMS1
